In [1]:
import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
import pandas as pd

from utils import predicted_test_data_to_result_csv
from keras import layers, losses, Input, Model
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Activation, Flatten, LSTM, SpatialDropout1D, Bidirectional
from keras.losses import sparse_categorical_crossentropy
from keras.metrics import sparse_categorical_accuracy
from keras.optimizers import Adam, SGD

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
train_path = "data/base/goodreads_train.csv"
result_path = "data/base/goodreads_test.csv"
frac_ratio = 0.2

In [5]:
max_features = 2000  # Maximum vocab size.
sequence_length = 200

In [6]:
df = pd.read_csv(train_path, sep=",")

In [7]:
index = df[(df['rating'] == 0)].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

In [8]:
x_train = df.sample(frac=frac_ratio)
x_val = df.drop(x_train.index)

In [9]:
y_train = x_train.pop('rating')
y_train = y_train - 1

y_val = x_val.pop('rating')
y_val = y_val - 1

In [10]:
x_train = x_train["review_text"]
x_val = x_val["review_text"]

In [11]:
raw_train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10, reshuffle_each_iteration=False)
raw_val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(10, reshuffle_each_iteration=False)

In [12]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_spoilers = tf.strings.regex_replace(lowercase, '\*\* spoiler alert \*\*', ' ')
    return tf.strings.regex_replace(stripped_spoilers,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [13]:
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [14]:
epochs = 20
model_nb = 4

embedding_dim = 100
learning_rate = 0.01
batch_size = 1024
dropout_rate = 0.2
hidden_layer = 3

In [15]:
raw_train_dataset = raw_train_dataset.batch(batch_size=batch_size)
raw_val_dataset = raw_val_dataset.batch(batch_size=batch_size)

In [16]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_dataset.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

val_text = raw_val_dataset.map(lambda x, y: x)
vectorize_layer.adapt(val_text)

In [17]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [18]:
train_ds = raw_train_dataset.map(vectorize_text)
val_ds = raw_val_dataset.map(vectorize_text)

In [19]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [20]:
input_text = Input(shape=(sequence_length,))

embedding_layer = Embedding(max_features + 1, embedding_dim, input_length=sequence_length)(input_text)

# spatial_dropout = SpatialDropout1D(dropout_rate)(embedding_layer)
# x = embedding_layer
# for i in range(hidden_layer):
#     x = SpatialDropout1D(dropout_rate)(x)
#     x = Bidirectional(LSTM(embedding_dim, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True))(x)
#
# lstm = Bidirectional(LSTM(25, return_sequences=False))(x)
x = embedding_layer
for i in range(hidden_layer):
    x = LSTM(embedding_dim, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True)(x)

lstm = LSTM(50, return_sequences=False)(x)

# global_max_pooling = GlobalMaxPooling1D()(lstm)
#
# dense = Dense(512, activation='relu')(lstm)

output = Dense(5, activation='softmax')(lstm)

rnn_model = Model(input_text, output)

rnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 100)          200100    
                                                                 
 lstm (LSTM)                 (None, 200, 100)          80400     
                                                                 
 lstm_1 (LSTM)               (None, 200, 100)          80400     
                                                                 
 lstm_2 (LSTM)               (None, 200, 100)          80400     
                                                                 
 lstm_3 (LSTM)               (None, 50)                30200     
                                                                 
 dense (Dense)               (None, 5)                 255   

In [21]:
rnn_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate=learning_rate),
                  metrics=sparse_categorical_accuracy)

In [22]:
exp_name = f'rnn_model_{model_nb}_hidden_layers_{hidden_layer}_lr_{learning_rate}_bs_{batch_size}_dr_{dropout_rate}'

In [23]:
rnn_model.fit(train_ds,
              validation_data=val_ds,
              epochs=epochs,
              callbacks=[tf.keras.callbacks.TensorBoard("logs/rnn/" + exp_name)])

Epoch 1/20
170/170 [==============================] - 994s 6s/step - loss: 1.3900 - sparse_categorical_accuracy: 0.3552 - val_loss: 1.3828 - val_sparse_categorical_accuracy: 0.3610
Epoch 2/20
170/170 [==============================] - 999s 6s/step - loss: 1.3808 - sparse_categorical_accuracy: 0.3610 - val_loss: 1.3832 - val_sparse_categorical_accuracy: 0.3610
Epoch 3/20
170/170 [==============================] - 992s 6s/step - loss: 1.3812 - sparse_categorical_accuracy: 0.3610 - val_loss: 1.3825 - val_sparse_categorical_accuracy: 0.3610
Epoch 4/20
170/170 [==============================] - 1024s 6s/step - loss: 1.3810 - sparse_categorical_accuracy: 0.3610 - val_loss: 1.3827 - val_sparse_categorical_accuracy: 0.3610
Epoch 5/20
170/170 [==============================] - 1024s 6s/step - loss: 1.3809 - sparse_categorical_accuracy: 0.3611 - val_loss: 1.3817 - val_sparse_categorical_accuracy: 0.3610
Epoch 6/20
170/170 [==============================] - 1027s 6s/step - loss: 1.2368 - sparse_c

In [24]:
df_test = pd.read_csv(result_path, sep=",")

df_test_modified = df_test.drop(columns=[
    'user_id',
    'book_id',
    'review_id',
    'date_added',
    'date_updated',
    'read_at',
    'started_at',
    'n_votes',
    'n_comments'
], inplace=False)

In [25]:
export_model = tf.keras.Sequential([
    vectorize_layer,
    rnn_model
])

In [26]:
predicted_test_data = export_model.predict(df_test_modified)

14939/14939 [==============================] - 8177s 547ms/step


In [27]:
df_test.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,b9450d1c1f97f891c392b1105959b56e,7092507,5c4df7e70e9b438c761f07a4620ccb7c,** spoiler alert ** \n This is definitely one ...,Sat Nov 10 06:06:13 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sat Nov 10 00:00:00 -0800 2012,1,0
1,b9450d1c1f97f891c392b1105959b56e,5576654,8eaeaf13213eeb16ad879a2a2591bbe5,"** spoiler alert ** \n ""You are what you drink...",Fri Nov 09 21:55:16 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,1,0
2,b9450d1c1f97f891c392b1105959b56e,15754052,dce649b733c153ba5363a0413cac988f,Roar is one of my favorite characters in Under...,Fri Nov 09 00:25:50 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,0,0
3,b9450d1c1f97f891c392b1105959b56e,17020,8a46df0bb997269d6834f9437a4b0a77,** spoiler alert ** \n If you feel like travel...,Thu Nov 01 00:28:39 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Thu Nov 01 00:00:00 -0700 2012,0,0
4,b9450d1c1f97f891c392b1105959b56e,12551082,d11d3091e22f1cf3cb865598de197599,3.5 stars \n I read and enjoyed the first two ...,Thu Oct 18 00:57:00 -0700 2012,Mon Apr 01 23:00:51 -0700 2013,Sat Mar 30 00:00:00 -0700 2013,Fri Mar 29 00:00:00 -0700 2013,0,0


In [28]:
predicted_test_data_to_result_csv(df_test, predicted_test_data, exp_name)